# Module 4: Matrices

In the next two modules, you will implement _matrix factorization_, a commonly used technique for _collaborative filtering_.

Your opinion about the movie Pulp Fiction and your opinion about the movie Reservoir Dogs are not two completely distinct opinions. The movies share so many similarities (director, actors, film style, genres, etc.) that most people who like one movie will also like the other. With matrix factorization, we aim to intelligently understand these opinions: instead of remembering a rating for Pulp Fiction and for Reservoir Dogs separately, we employ a mathematical trick that encodes users' movie preferences more efficiently.

The elegance of matrix factorization lies in its generality, as it has far more applications beyond recommender systems alone. You will likely encounter this technique (or at least aspects/variants) more frequently in the future.

To implement matrix factorization, we need to cover several fundamental aspects of machine learning and linear algebra. This module primarily focuses on linear algebra, while the following module centers around the necessary machine learning concepts.

This module consists of five parts:
- Part 1: Matrix Transposition
- Part 2: Vector Dot Product
- Part 3: Matrix Dot Product
- Part 4: Mean Square Error
- Part 5: Other Matrix Operations

## Packages and Data
Start by loading the required packages below.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from helpers import nice, random_df, display_side_by_side
import answers

data_folder = './data/'

Load the table below:

In [ ]:
pre_defined_items = pd.read_csv(f'{data_folder}three_genres_movies.csv', index_col = 'movieName')
display(nice(pre_defined_items))

This table contains 19 movies. For each movie, affiliation with certain genres is shown on a scale with a maximum of 1 and a minimum of 0. As you can see, _Pocahontas_ scores high for the _drama_ genre, _Inception_ scores high for both _drama_ and _thriller_, and _South Park: Bigger, Longer and Uncut_ is primarily a _comedy_ movie.

In this case, genres are not encoded as categorical data but as numerical data. In other words, a movie can belong to a genre to a greater extent (1) or a lesser extent (0).

For this assignment, we'll focus on three genres (_drama_, _thriller_, and _comedy_). Using this approach, we keep things relatively manageable. In reality, you'd generally use many more genres.

<table style= "width:100%"><tr><td style="border:1px solid black">
    
Note that we use the `nice` function to display the data. This provides the color coding you see in the table, which will be particularly useful later in this module to maintain clarity.
    
</td></tr></table>

We've created a similar table for the users. The table below provides information about which genres each user generally likes.

For this table, a score has been assigned for each user and each genre as well. In this case, the scores range between -1 and 1. If the value for a particular genre is negative for a specific user, it means that the user prefers not to watch movies of that genre.

In [ ]:
pre_defined_users = pd.read_csv(f'{data_folder}three_genres_users.csv', index_col = 'userId')

display(nice(pre_defined_users))

### Matrices

We can consider these tables as **matrices**. These matrices characterize the genres of movies and the preferences of users. The formal definition of matrices in linear algebra is more complex, but in this assignment, we can simply think of a matrix as a table of numbers. All the `DataFrame`s we use in this assignment are matrices.

In linear algebra, it's common to denote matrices with capital letters. For all the equations provided in the remainder of this module, we will refer to the movie matrix above as $M$ (movie matrix) and the user matrix above as $U$ (user matrix).

### Dimensions

It's often crucial to keep track of the dimensions of matrices when working with them. The matrix $M$ above contains 19 rows (19 movies) and 3 columns (3 genres). In other words, the dimensions of $M$ are $19 \times 3$. Thus, the matrix $U$ has dimensions of $30 \times 3$.

## Part 1: Matrix Transposition

The first operation we need to learn is matrix transposition. Its significance will become clear later.

Transposing a matrix is simply swapping its columns and rows. For example, the transposed version $U^T$ of the user matrix $U$ appears as follows:

![](src/users_transposed.png)

Just like in the original table, a score is assigned per genre for each user. However, the dimensions of the transposed matrix are flipped: $U$ is a $30 \times 3$ matrix (30 rows and 3 columns), and $U^T$ is a $3 \times 30$ matrix (3 rows and 30 columns).

We denote the transpose of a matrix $X$ as $X^T$.

In general, if we have a matrix $X$:

$$
X =
\begin{pmatrix}
a_1    & b_1    & \cdots \\
a_2    & b_2    & \cdots \\
a_3    & b_3    & \cdots \\
\vdots & \vdots & \ddots
\end{pmatrix}
$$

the transpose of that matrix $X^T$ looks like this:

$$
X^T = 
\begin{pmatrix}
a_1    & a_2    & a_3    & \cdots \\
b_1    & b_2    & b_3    & \cdots \\
\vdots & \vdots & \vdots & \ddots
\end{pmatrix}
$$

Thus, the rows of $X$ become the columns of $X^T$.

### Question 1

\[2 pts.\]

Write the function `transpose` below, which takes a matrix as input (in this case, a Pandas `DataFrame`) and outputs its transposed version. The goal is to write this function using a `for` loop. So, you're not allowed to use the built-in transpose function in Pandas (`df.T`) for now. Also, make sure that the labels of the columns and rows switch: the labels of the input rows become the labels of the output columns, and vice versa.

In [ ]:
def transpose(matrix: pd.DataFrame) -> pd.DataFrame:
    # TODO
    

Try out some examples below to see if your code works as expected.

In [ ]:
# Example 1: the user matrix

pre_defined_users_transposed = transpose(pre_defined_users)
display(nice(pre_defined_users_transposed))

# Try some more examples

# TODO


Use the test provided below to check if your implementation is correct:

In [ ]:
answers.test_1(pre_defined_users_transposed, pre_defined_users)

### 🏁 Checkpoint

Many operations from linear algebra are already built into Pandas. For calculating the transpose of the matrix `my_df`, you can simply write:

`my_df_transposed = my_df.T`

From now on, you're allowed to use this instead of your own transpose function.

## Part 2: Vector Dot Product

The next concept from linear algebra that we need is the **vector dot product**.

Ultimately, we want to know how well a specific user will like a particular movie. So, we want to predict a rating for the combination of a user and a movie.

### Example 1

Suppose we want to predict the rating of user 13 for the movie Inception. We can use the _dot product_ for that.

The row for Inception in the user matrix contains the scores for that movie.

<img src="src/movies_select_inception.PNG" width=30%>

We can consider this row as a vector:

$$
m_{\textrm{inception}} = (0.63, 0.92, 0.44)
$$

The first element indicates that the movie scores 0.63 for the _drama_ genre, the second element indicates that the movie scores 0.92 for the _thriller_ genre, and the third element indicates that the movie scores 0.44 for the _comedy_ genre.

<table style="width:100%"><tr><td style="border:1px solid black">
    
A vector is not the same as a matrix. A vector has a precise definition in linear algebra, but for now, we can define a vector as a list of numbers. In this notebook, we implement a vector as a Pandas `Series`.
    
</td></tr></table>

We can extract the user 13 scores from the column of the transposed user matrix in the same way:

<img src="src/users_select_13.PNG" width=40%>

We can also interpret this column as a vector:

$$
u_{13} = (0.75, 0.75, 0.29)
$$

This user has scores of 0.75 for both _drama_ and _thriller_ genres and a score of 0.29 for the _comedy_ genre.

Now we can calculate the predicted rating by multiplying the scores of Inception by the user's scores for each genre. We do this for each genre and sum up the results:

$$
m_{\textrm{inception, drama}} \cdot u_{13,\textrm{ drama}} + m_{\textrm{inception, thriller}} \cdot u_{13,\textrm{ thriller}} + m_{\textrm{inception, comedy}} \cdot u_{13,\textrm{ comedy}} = 0.63 \cdot 0.75 + 0.92 \cdot 0.75 + 0.44 \cdot 0.29 = 1.2901
$$

Therefore, the predicted rating for user 13 for Inception is $1.2901$. We write this dot product as follows:

$$ 
m_{\textrm{inception}} \cdot u_{13} = 1.2901
$$

<table style="width:100%"><tr><td style="border:1px solid black">
    
By the way, this is not a rating on a scale of 0 to 5, as in previous assignments. The score is on a scale of 0 to 1, where anything above 1 is considered 1, and anything below 0 is considered 0. A score of 1.29 is well above the maximum rating.

</td></tr></table>

### Definition

In general, if we have two vectors $a = (a_1, a_2, \ldots, a_n)$ and $b = (b_1, b_2, \ldots, b_n)$, the **dot product** is defined as follows:

$$
a \cdot b = a_1\cdot b_1 + a_2\cdot b_2 + \ldots + a_n\cdot b_n
$$

### Example 2

If we use the movie Dumb & Dumber instead of Inception, we get the following vector $m_{\textrm{dumb\&dumber}} = (0.00, 0.17, 1.00)$. If we want to know the rating of user 13 for Dumb & Dumber, we can again use the dot product:

$$
m_{\textrm{dumb\&dumber}} \cdot u_{13} = 0.00 \cdot 0.75 + 0.17 \cdot 0.75 + 1.00 \cdot 0.29 = 0.4175
$$

The predicted rating for Dumb & Dumber is much lower than that for Inception. This isn't surprising, given that Dumb & Dumber scores significantly for the _comedy_ genre (1.00), while the user has a relatively low score for that genre (0.29). In other words, Dumb & Dumber is primarily a _comedy_ movie, but the user clearly doesn't enjoy _comedy_ much.

### Limitation

It's important to realize that the above approach only works because the number of genres for users matches the number of genres for movies. For example, if we have 4 genres for movies and only 2 for users, the above logic wouldn't work because we can't compute a dot product between a vector with 2 elements and a vector with 4 elements. If:

$$
m_{\textrm{inception}} = (0.63, 0.92, 0.44, 0.18)
$$

and

$$
u_{13} = (0.75, 0.75)
$$

Then the dot product, $m_{\textrm{inception}} \cdot u_{13} = 0.63 \cdot 0.75 + 0.92 \cdot 0.75 + 0.44 \cdot \mathbf{??} + 0.18 \cdot \mathbf{??} = \mathbf{????}$, is undefined.

In other words, the dot product between two vectors is only defined when both vectors have the same length.

### Question 2

\[2 pts.\]

Write the function `dot_v` below. This function calculates the dot product of two vectors (Pandas `Series`). It should work for two `Series` of arbitrary (but equal) length.

You're not allowed to use the `@`-operator or a built-in function here, even when you're already familiar. You need to implement the product manually using a loop.

In [ ]:
def dot_v(v1: pd.Series, v2: pd.Series) -> float:
    # TODO
    

# get row for 'Inception' from film matrix
m_inception = pre_defined_items.loc['Inception']

# get column for 13 from the transposed user matrix 
u_13 = pre_defined_users_transposed.loc[:,13]

# show vectors (pd.Series)
print(m_inception)
print(u_13)
print()

# compute dot product between the two vectors
print(f'Predicted rating: {dot_v(m_inception, u_13)}')

Test your implementation:

In [ ]:
answers.test_2(dot_v)

## Part 3: Matrix Dot Product

Above, we've seen how to use the _vector_ dot product to calculate a predicted rating for a combination of a movie and a user.

Like with the other algorithms you've written, we want to create a utility matrix. In this case, the matrix contains the _predicted ratings_ for _all_ combinations of movies and users. The rows of the matrix represent the movies, and the columns represent the users.

If we only look at the first 10 x 10 elements, the matrix looks approximately like this:

<img src = "src/utility.PNG" width = 50%>

For example, you can see that the predicted rating of user 13 for the movie Inception is about 1.29, as we calculated above.

We'll call this matrix of predicted ratings $\hat{R}$. The shown part of the matrix above is small. In reality, $\hat{R}$ has dimensions of $19 \times 30$: 19 rows (19 movies) and 30 columns (30 users).

To calculate this matrix, we can use the _vector_ dot product (`dot_v` function) on each possible combination of user and movie. If we zoom in on the first three users and the first two movies, we calculate the utility matrix as follows:

$$
\begin{array}
.                  & 6                               & 7                               & 8                               & \cdots\\ 
\textrm{Babe}      & m_\textrm{babe}\cdot u_{6}      & m_\textrm{babe}\cdot u_{7}      & m_\textrm{babe}\cdot u_{8}      & \cdots\\
\textrm{Inception} & m_\textrm{inception}\cdot u_{6} & m_\textrm{inception}\cdot u_{7} & m_\textrm{inception}\cdot u_{8} & \cdots\\
\hspace{1.5em}\vdots & \hspace{1.5em}\vdots & \hspace{1.5em}\vdots & \hspace{1.5em}\vdots & \ddots
\end{array}
$$

For example, let's look at the first element in the matrix: the predicted rating for user 6 for the movie Babe. This is given by the vector dot product: $m_\textrm{babe}\cdot u_{6}$.

<table style= "width:100%"><tr><td style="border:1px solid black">
    
In concrete terms, this gives the following result:

The value for the vector $m_\textrm{babe}$ can be taken from the matrix with movies. This is $m_\textrm{babe} = (0.74, 0.22, 0.43)$ (so, the movie Babe has a score of $0.74$ for the genre _drama_, $0.22$ for _thriller_, and $0.43$ for _comedy_). 

The value for the vector $u_{6}$ can be taken from the matrix with users.
  This gives $u_{6} = (0.71, -0.65, 0.56)$ (so, user 6 has a score of $0.71$ for the genre _drama_, $-0.65$ for _thriller_, and $0.56$ for _comedy_). 

With these values, we can calculate the vector dot product: $m_\textrm{babe}\cdot u_{6} = 0.74 \cdot 0.71 + 0.22 \cdot -0.65 + 0.43 \cdot 0.56 = 0.62$
</td></tr></table>

If we do this for all fields of the utility matrix, that is, all combinations of movies and users, then we have calculated the **matrix dot product** between the user matrix and the movie matrix.

More formally, if $M$ is the matrix with movies and $U^T$ is the transposed user matrix, the utility matrix with the predicted ratings ($\hat{R}$) is the dot product of $M$ and $U^T$. We write this as follows:

$$
\hat{R} = M \cdot U^T
$$

Note that for this definition of the user matrix, we take its transposed version ($U^T$). In linear algebra, the matrix dot product is defined in such a way that the *rows* of the result (i.e., the movies in matrix $\hat{R}$) must match the *rows* in the left component of the multiplication ($M$). And the *columns* of the result (i.e., the users in matrix $\hat{R}$) must match the *columns* of the right-side component of the multiplication ($U^T$).

As a reminder, the dimensions of $M$ are $19 \times 3$ (19 rows/movies and 3 columns/genres), and the dimensions of $U^T$ are $3 \times 30$ (3 rows/genres and 30 columns/users). The result of the matrix multiplication, $\hat{R}$, has dimensions $19 \times 30$.

More generally, if you have a $k \times l$ matrix $A$ and an $m \times n$ matrix $B$, the dot product $A \cdot B$ will have dimensions $k \times n$.

It's also important to realize that the above calculation of $\hat{R}$ worked because the number of genres for users matched the number of genres for movies. If, for example, we know 4 genres for movies and 2 for users, the above logic wouldn't work, as we've seen that we can only calculate a dot product between vectors of the same length.

This indicates that the dot product of a $k \times l$ matrix $A$ and an $m \times n$ matrix $B$ is only defined if $l = m$.

So, for the calculation of $\hat{R}$, everything works fine, since $l=m={\rm the\ number\ of\ genres} = 3$.

### Question 3

The task below is to implement the `dot` function, which calculates the matrix dot product. The DataFrame `output` is already defined with the correct dimensions, but it's currently filled with zeros. Write code that fills the `output` with the correct values.

You're not allowed to use the `@`-operator or a built-in function here, even when you're already familiar.

<table style= "width:100%"><tr><td style="border:1px solid black">
    
Tip 1: You have to write a nested loop that iterates over the rows of matrix 1 and the columns of matrix 2.

Tip 2: Inside the nested loop, you can call the `dot_v` function from question 2 to calculate the predicted rating.
</td></tr></table>

In [ ]:
def dot(matrix1: pd.DataFrame, matrix2: pd.DataFrame) -> pd.DataFrame:
    if not matrix1.shape[1] == matrix2.shape[0]:
        raise Exception(f'Cannot multiply matrices of dimensions {matrix1.shape} and {matrix2.shape}')
    output = pd.DataFrame(0, index = matrix1.index, columns = matrix2.columns)
    
    # TODO
    
    return output


predicted = dot(pre_defined_items, pre_defined_users_transposed)

# Show the first 10x10 values:
display(nice(predicted.iloc[:10,:10]))

Test your implementation:

In [ ]:
answers.test_3(predicted)

### 🏁 Checkpoint

Pandas has also built-in the matrix dot product, not just in the form of a function but also as an operator: `@`. The matrix dot product $X = A \cdot B$ can be calculated using Pandas like this:

	X = A @ B

From now on, you're allowed to use the `@` operator to calculate the matrix dot product. We can calculate the `predicted` matrix using this operator as follows:

In [ ]:
predicted = pre_defined_items @ pre_defined_users_transposed
display(nice(predicted.iloc[:10,:10]))

## Part 4: Mean Squared Error

Now we've seen how to generate a utility matrix with predictions. In doing so, we used pre-defined matrices with genre scores per user and per film. However, you won't always have these pre-defined matrices available. In the end, what want to do is to generate (learn) the matrices for users and films based on the available data. For this, we need a bit of machine learning.

But before diving into that, let's first determine how accurate our current predictions are. We'll use the _MSE_ (_mean squared error_).

First, we'll load some review data:

In [ ]:
ratings = pd.read_csv(f'{data_folder}three_genres_ratings.csv', index_col = 'movieName', dtype={'userId': np.int64})
display(ratings)

This data contains ratings that users have given for films. The data is normalized so that the highest rating is $1$ and the lowest rating is $0$.

Now, we can use the _MSE_ to assess how well the predictions align with this data. We're going to approach this slightly differently than before. To calculate the _MSE_, we first convert the data into matrix form:

In [ ]:
target = ratings.pivot(columns = 'userId', values = 'value').loc[predicted.index, predicted.columns]

display(nice(target.iloc[:15,:15], nan = '&middot;'))

Not all combinations of films and users have a rating. The combinations for which no value is known are indicated by nan values. To keep the presentation clear, all nan values are represented as dots ($\cdot$).

Below, we calculate the MSE for our predicted matrix predicted, given the data target. The MSE function below ignores all nan values. So, when calculating the MSE, only the combinations of films and users for which we have information in our data are considered.

<table style= "width:100%"><tr><td style="border:1px solid black">

The function below subtracts two matrices from each other (`m2 - m1`). We will discuss how this process works in the final part of this module.
</td></tr></table>

In [ ]:
def MSE(m1, m2):
    se = (m2 - m1)**2
    return se.mean().mean()

print(MSE(target, predicted))

Is this a good score? Let's first take a look at how well this compares to random guessing:

In [ ]:
random_prediction = random_df(target.index, target.columns)

print(MSE(target, random_prediction))

The `random_prediction` results in a much higher MSE than our prediction. That's not surprising, because the user and movie matrices we used for our predictions were already quite optimized.

## Part 5: Other Matrix Operations

### Scalar Multiplication

As mentioned, our end goal is to generate (learn) matrices for the users and movies based on the available data. We will explore this in the next module. However, before we can do so, we need to introduce several other matrix operations.

These operations are not directly related to movie ratings (unlike the previous operations). Their significance will become clear in the next module. For now, let's briefly understand how they work.

You can also multiply matrices by a single number ($\alpha$). In this case, you multiply each number in the matrix by $\alpha$:

$$
2 \cdot 
\begin{pmatrix}
3    & 4      \\
1    & 2      \\
0    & 4      \\
\end{pmatrix}
= 
\begin{pmatrix}
2\cdot3    & 2\cdot4      \\
2\cdot1    & 2\cdot2      \\
2\cdot0    & 2\cdot4      \\
\end{pmatrix}
=
\begin{pmatrix}
6    & 8      \\
2    & 4      \\
0    & 8      \\
\end{pmatrix}
$$

In general, given number $\alpha$ and matrix $X$:

$$
X =
\begin{pmatrix}
a_1    & b_1    & \cdots \\
a_2    & b_2    & \cdots \\
a_3    & b_3    & \cdots \\
\vdots & \vdots & \ddots
\end{pmatrix}
$$

scalar multiplication (multiplying a number by a matrix) $\alpha \cdot X$ is defined as follows:

$$
\alpha \cdot X =
\begin{pmatrix}
\alpha \cdot a_1    & \alpha \cdot b_1    & \cdots \\
\alpha \cdot a_2    & \alpha \cdot b_2    & \cdots \\
\alpha \cdot a_3    & \alpha \cdot b_3    & \cdots \\
\vdots & \vdots & \ddots
\end{pmatrix}
$$

In Pandas, you can perform scalar multiplication using the `*` operator:

In [ ]:
a = 2
X = pd.DataFrame([[3,4],[1,2],[0,4]])
M = a * X
display(M)

## Matrix Addition

You can also add two matrices together:

$$
\begin{pmatrix}
3    & 4      \\
1    & 2      \\
0    & 4      \\
\end{pmatrix}
+
\begin{pmatrix}
0    & 0.5      \\
1    & 2      \\
1    & 1      \\
\end{pmatrix}
=
\begin{pmatrix}
3+0    & 4+0.5      \\
1+1    & 2+2      \\
0+1    & 4+1      \\
\end{pmatrix}
=
\begin{pmatrix}
3    & 4.5     \\
2    & 4      \\
1    & 5      \\
\end{pmatrix}
$$

In general, if we have two matrices $X$ and $Y$:

$$
X =
\begin{pmatrix}
a_1    & b_1    & \cdots \\
a_2    & b_2    & \cdots \\
a_3    & b_3    & \cdots \\
\vdots & \vdots & \ddots
\end{pmatrix}
,
Y =
\begin{pmatrix}
i_1    & j_1    & \cdots \\
i_2    & j_2    & \cdots \\
i_3    & j_3    & \cdots \\
\vdots & \vdots & \ddots
\end{pmatrix}
$$

Then $X+Y$ is:

$$
X + Y =
\begin{pmatrix}
a_1    & b_1    & \cdots \\
a_2    & b_2    & \cdots \\
a_3    & b_3    & \cdots \\
\vdots & \vdots & \ddots
\end{pmatrix}
+
\begin{pmatrix}
i_1    & j_1    & \cdots \\
i_2    & j_2    & \cdots \\
i_3    & j_3    & \cdots \\
\vdots & \vdots & \ddots
\end{pmatrix}
=
\begin{pmatrix}
a_1+i_1    & b_1+j_1    & \cdots \\
a_2+i_2    & b_2+j_2    & \cdots \\
a_3+i_3    & b_3+j_3    & \cdots \\
\vdots & \vdots & \ddots
\end{pmatrix}
$$

This only works if $X$ and $Y$ have the same dimensions (the same number of columns and rows). You can similarly subtract one matrix from another in the same way.

In Pandas, you can simply use the `+` and `-` operators for adding and subtracting matrices:

In [ ]:
a = 2
X = pd.DataFrame([[3,4],[1,2],[0,4]])
Y = pd.DataFrame([[0,0.5],[1,2],[1,1]])
M = X + Y
display(M)

### Question 4

\[1 pt.\]

Use the matrix operators `+`, `-`, `*`, and `@` to compute the following equations in Pandas:

$$
M = a \cdot (X \cdot Y^T) - 2 \cdot X
$$

The values for $X$, $Y$, and $a$ are shown below.

In [ ]:
a = 2
X = pd.DataFrame([[3,4],[1,2],[0,4]])
Y = pd.DataFrame([[0,1],[1,2]])

# TODO

display(M)

Test your implementation:

In [ ]:
answers.test_4(M)

### Question 5

\[2 pt.\]

Multiplying $3 \times 6$ is the same as $6 \times 3$. Generally, for numbers, multiplication is commutative. This means that $a \times b$ is the same as $b \times a$.

Is matrix multiplication also commutative? In other words, for two matrices $A$ and $B$, is the product $A \cdot B$ the same as $B \cdot A$? Why or why not?

YOUR ANSWER HERE

## Finished

You have learned how to multiply matrices and how to use that to predict ratings based on genres. You have seen how to use the mean squared error (MSE) to evaluate the quality of those predictions. Additionally, you have learned how scalar multiplication and addition and subtraction work for matrices. With this, you have covered enough linear algebra for the next module.

In the next module, you will learn how to learn the matrices with genres using given data. For this purpose, you will learn about a crucial machine learning technique: _gradient descent_ (and a variation known as _alternating least squares_).